In [55]:
import time
import random 

import numpy as np
from tqdm import tqdm
from bs4 import BeautifulSoup

import click

from whyclick.chrome import open_chrome, remove_popups
from whyclick import whyq

import torch

from collections import Counter

In [56]:
query = {'username': "*****", 'password': "*****"}

driver = whyq.login(query['username'], query['password'], headless=False)

chromedriver is already installed.


In [ ]:
order_json = whyq.download_previous_orders(driver)

In [5]:
ordered_items = [o['Item Name'] for o in order_json]

In [6]:
ordered_items[-10:]

['Tonkatsu Set (Chicken Cutlet) (Japanese Food & Salad - MSI)',
 'Balinese Gyudon Egg Bowl (Rayyan – Amoy)',
 'Hor Fun (Healthy Vegetarian-Amoy)',
 'Pulled Pork Quesadilla (James Quesadilla & Brunch -Amoy)',
 'Moussaka (Avail Thurs/Fri)(Fill-a-pita) (Fill-a-pita-Standalone)',
 "Pasta Aglio-Olio Mushroom (M)  (Chef'B Western - Amoy)",
 'Fish & Chips (Chef B Western - Amoy)',
 'Homemade Hamburger (Beef)(Zipp Burger & Pasta-Amoy)',
 'Char Siew Biryani Bento  (Chop Chop Biryani & Meats-Amoy)',
 'Crispy Chicken Chop Rice (Chikedap - Amoy)']

In [7]:
from transformers import DistilBertTokenizer, DistilBertModel, DistilBertForSequenceClassification

In [8]:
tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
model = DistilBertModel.from_pretrained('distilbert-base-uncased')

In [9]:
def vectorize(text):
    input_ids = torch.tensor(tokenizer.encode(text)).unsqueeze(0)
    ff = model(input_ids)
    output = ff[0].squeeze().squeeze()[0]
    return output.detach().numpy()


In [10]:
# Converts sentences to arrays of floats.
vectorized_sents = [vectorize(s) for s in tqdm(ordered_items)]
vectorized_sents = torch.tensor(np.array(vectorized_sents))

100%|██████████| 98/98 [00:06<00:00, 15.30it/s]


In [11]:
driver.get('http://whyq.sg')

In [77]:
def smartly_order_one_day(driver, element_day, simple_max=False, halal=False, healthy=False, vegetarian=False):
    time.sleep(1)
    loop_count = 0 # Sanity break.
    global tempered_vectorized_sents
    while True:
        try:
            # Apply dietary filter
            whyq.apply_dietary_filters(driver, halal, healthy, vegetarian)
            # Find meals.
            meals = [b for b in element_day.find_elements_by_xpath('//button')
                     if b and b.text == "ADD"]

            meals_str = [h6.text for h6 in element_day.find_elements_by_xpath('//h6')][:len(meals)]
            meals_vec = torch.tensor(np.array([vectorize(m) for m in meals_str]))
            
            # For every meal choice, I find what's most similar, by scores with temperature. 
            sim_scores = torch.tensor(np.dot(meals_vec, tempered_vectorized_sents.T))
            if simple_max:
                """ Old neural top-1 choice.
                top_m, top_m_score = -1, -1
                for i, m in enumerate(meals_vec):
                    for s in vectorized_sents:
                        if np.dot(m, s.T) > top_m_score:
                            top_m = i
                """
                top_m = int(torch.max(torch.max(sim_scores, dim=1).values, dim=0).indices)
            else:
                # For every meal choice, pick from a multinomial distribution.
                top_m = int(torch.multinomial(torch.max(sim_scores, dim=1).values, 1).view(-1))
            
            meals[top_m].click()

            # Randomly choose one.
            ##random.choice(meals).click()

            # Check if you've ordered already.
            time.sleep(0.3)
            msg = element_day.find_element_by_xpath('//div[@id="notify_msg"]')
            break
        except IndexError: # No meals from dietary restriction.
            # Repeat the loop so that the filters are undone.
            pass
        if loop_count > 3: # Sanity break.
            break # If everything fails, go to next day.
        loop_count += 1
    return driver, element_day, msg.text

In [78]:
temperature = 0.7
tempered_vectorized_sents = vectorized_sents.div(temperature).exp().cpu()
#tempered_vectorized_sents = vectorized_sents

days = driver.find_elements_by_xpath("//div[@class='owl-item active']")

for element_day in days:
    element_day.click()
    driver, element_day, msg = smartly_order_one_day(
        driver, element_day, #simple_max=True
    )
    time.sleep(3)


In [79]:
driver.find_element_by_link_text("PLACE ORDER").click()